# 物体检测与区域卷积神经网络
CNN卷积神经网络主要用于物体分类，物体检测不但包括物体分类，还要包括物体在图片中的定位

#1 ILSVRC竞赛
包括物体检测、图像分类、场景分类、物体定位、场景解析

#2 区域卷积神经网络R-CNN
物体检测主要分为两个步骤
- 区域建议
- 区域分类

也就是说首先先定位出有多少个物品，以及每个物品的区域，然后抠出小图，传入CNN中进行物体分类，具体的网络进化包括$RCNN\rightarrow{SPPNET}\rightarrow{FastRCNN}\rightarrow{FasterRCNN}\rightarrow{RFCN}$

#3 R-CNN(Regions with CNN)网络


##3.1 包含模块
1. 模块一：Selective Search(SS)算法获取区域，跟分类无关，包含物体
2. 做区域预处理，将上一步的结果进行Bounding box膨胀,做16个像素的膨胀，尺寸变味227*227
3. 模块二:送入AlexNet网络，对所有区域进行特征提取(从第七层，也就是全连接的第二层)，还需要做fine-tune
4. 模块三:送入线性SVM分类器，对CNN特征(4096)进行分类，每个分类一个2分类SVM
5. 模块四:Bounding box回归模型，对SS提供的区域进行精化，当然需要基于CNN的特征，每个分类一个回归模型


##3.2 IoU
物体检测需要定位出物体的Bounding box，并且需要识别处bounding box里面的物体就是车辆。因为我们的散发不可能百分百跟人工标注的数据完全匹配，因此就存在一个定位精度评价公式IoU。IoU定义了两个bounding box的重叠度，这个重叠就是$A\cap{B}$。重叠度IoU的计算公式是:$IoU=\frac{A\cap{B}}{A\cup{B}}$，也就是重叠面积占A、B并集面积的比例:$IOU=\frac{S_I}{S_A+S_B-S_I}$

##3.3 非极大值抑制
检测的结果会从一张图片中找出n多个可能是物体的矩形框，然后为每个矩形框做类别分类概率，比如先假设有6个矩形框，根据分类器类别分类概率做排序，从小到大分别属于车辆的概率分别为A、B、C、D、E、F:
- 从最大概率矩形框F开始，分别判断A~E与F的重叠度IOU是否大于某个设定的阈值
- 假设B、D与F的重叠度超过阈值，那么就扔掉B、D；并标记第一个矩形框F，是我们保留下来的。
- 从剩下的矩形框A、C、E中，选择概率最大的E，然后判断E与A、C的重叠度，重叠度大于一定的阈值，那么就扔掉；并标记E是我们保留下来的第二个矩形框。
- 就这样一直重复，找到所有被保留下来的矩形框

##3.4 总体思路
首先要明确的是图片分类与物体检测不同，物体检测需要定位出物体的位置，相当于回归问题，求解一个包含物体的方框。因此方法就是：首先输入一张图片，我们先定位出2000个物体候选框，然后采用CNN提取每个候选框中图片的特征向量，特征向量的维度为4096维，接着采用svm算法对各个候选框中的物体进行分类识别。也就是总个过程分为三个程序：
- 找出候选框
- 利用CNN提取特征向量
- 利用SVM进行特征向量分类

##3.5 实现方式

###3.5.1 寻找物体区域
采用的方法就是Selective Search算法，这个算法可以搜索处2000个候选框，而且都是矩形且大小各不相同。但是CNN对于输入的图片大小是有大小限制的，所以在放入CNN之前，需要对于每个输入的候选框都需要缩放到固定的大小
- 各向异性缩放:这种方法很简单，就是不管图片的长宽比例，管它是否扭曲，进行缩放就是了，全部缩放到CNN输入的大小227*227，如下图(D)所示- 各向同性缩放:因为图片扭曲后，估计会对后续CNN的训练精度有影响，这个有两种办法
	- 直接在原始图片中，把bounding box的边界进行扩展延伸成正方形，然后再进行裁剪；如果已经延伸到了原始图片的外边界，那么就用bounding box中的颜色均值填充；如下图(B)所示;
	- 先把bounding box图片裁剪出来，然后用固定的背景颜色填充成正方形图片(背景颜色也是采用bounding box的像素颜色均值),如下图(C)所示;![images](images/12/35.png)对于上面的异性、同性缩放，文献还有个padding处理，上面的示意图中第1、3行就是结合了padding=0,第2、4行结果图采用padding=16的结果。经过最后的试验，作者发现采用各向异性缩放、padding=16的精度最高.上面处理完后，可以得到指定大小的图片，因为我们后面还要继续用这2000个候选框图片，继续训练CNN、SVM

##3.2 训练流程
1. M$\leftarrow$在ImageNet上对CNN模型进行pre-train
2. M'$\leftarrow$使用所有SS生成区域对M进行fine-tune，这一步仍然是使用AlexNet的东西来进行训练，只不过真正的AlexNet得到的是1000个分类，而真正的竞赛分类可能是20，或者200，然后加上一个背景分类，所以需要将1000个分类映射到真正的分类上
	- Loss使用的是AlexNet的概率log值
	- Softmax层改成(N+1)维，其余不变
	- 32个正样本(N类)
	- 96个负样本(1类)
3. C$\leftarrow$在M'的FC7(也就是全连接第二层)特征熵训练线性SVMs分类器
	- Hinge Loss
	- 每个类别(N类)对应一个SVM分类器
	- 正样本:所有Ground-truth区域
	- 负样本:跟Ground-truth重合IoU<0.3的SS区域
4. R$\leftarrow$在M'的Conv5特征上训练Bounding box回归模型
	- 提升定位性能(Bounding box的准确性)
	- 每个类别(N类)训练一个回归模型
		- 将SS提供的Bounding box做重新映射$P\rightarrow{G}$
		- 训练输入
		- P的IoU > 0.6
		- Squared loss
	- 测试阶段
		- 参数w已经训练好
		
如果检测出来的区域跟Ground-truth重合度IoU >= 0.5，那么就是正样本，反之就是负样本

pre-train:预训练，拿一个已经训练好的模型，作为初始模型
fine-tune:对于预训练的模型，使用当前的训练集继续训练